In [0]:
ipl_matches_df.select("match_name").filter(col("match_name").isNotNull()).show(50, truncate=False)


+----------------------------------------------------------------+
|match_name                                                      |
+----------------------------------------------------------------+
|MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match        |
|CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match                |
|SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, 3rd Match         |
|RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match                     |
|KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th Match              |
|SUNRISERS HYDERABAD vs ROYAL CHALLENGERS BENGALURU, 6th Match   |
|RAJASTHAN ROYALS vs DELHI CAPITALS, 7th Match                   |
|PUNJAB KINGS vs CHENNAI SUPER KINGS, 8th Match                  |
|MUMBAI INDIANS vs SUNRISERS HYDERABAD, 9th Match                |
|ROYAL CHALLENGERS BENGALURU vs KOLKATA KNIGHT RIDERS, 10th Match|
|DELHI CAPITALS vs PUNJAB KINGS, 11th Match                      |
|CHENNAI SUPER KINGS vs RAJASTHAN ROYALS, 12th Match          

In [0]:
df = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/ipl_2021_matches.csv")
df.show(5)

+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|        T20|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|https://www.cricb...|       null|       null|     

In [0]:
mock_tied_match = [Row(
    year="2021",
    series_type="T20",
    series_name="Indian Premier League 2021",
    match_no="50",
    match_type="Playoff",
    match_name="DELHI CAPITALS vs RAJASTHAN ROYALS, Tied Match (Super Over)",
    match_href="https://www.cricbuzz.com/cricket-scores/99999/dc-vs-rr-tied-match-super-over",
    match_team1="DELHI CAPITALS",
    match_team2="RAJASTHAN ROYALS",
    match_datetime_start="2021-05-10T19:30:00",
    match_date_end="2021-05-10",
    match_venue="Wankhede Stadium"
)]


In [0]:
mock_df = spark.createDataFrame(mock_tied_match)


In [0]:
combined_df = ipl_matches_df.unionByName(mock_df)

In [0]:
tied_matches_df = combined_df.filter(
    lower(col("match_name")).like("%tie%") | lower(col("match_name")).like("%super over%")
)


In [0]:
tie_breakers_df = tied_matches_df.select("match_name", "match_href", "match_venue")
tie_breakers_df.show(truncate=False)

+-----------------------------------------------------------+----------------------------------------------------------------------------+----------------+
|match_name                                                 |match_href                                                                  |match_venue     |
+-----------------------------------------------------------+----------------------------------------------------------------------------+----------------+
|DELHI CAPITALS vs RAJASTHAN ROYALS, Tied Match (Super Over)|https://www.cricbuzz.com/cricket-scores/99999/dc-vs-rr-tied-match-super-over|Wankhede Stadium|
+-----------------------------------------------------------+----------------------------------------------------------------------------+----------------+



In [0]:
df.select("match_name").show(5, truncate=False)


+--------------------------------------------------------+
|match_name                                              |
+--------------------------------------------------------+
|MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match|
|CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match        |
|SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, 3rd Match |
|RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match             |
|KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th Match      |
+--------------------------------------------------------+
only showing top 5 rows



In [0]:
tie_breaker_matches = df.filter(
    df.match_name.contains("Final") | 
    df.match_name.contains("Qualifier") | 
    df.match_name.contains("Eliminator")
)
tie_breaker_matches = tie_breaker_matches.select("match_name", "match_type").distinct()
tie_breaker_matches.show(truncate=False)



+----------------------------------------------------------------+----------+
|match_name                                                      |match_type|
+----------------------------------------------------------------+----------+
|CHENNAI SUPER KINGS vs KOLKATA KNIGHT RIDERS, Final             |League    |
|DELHI CAPITALS vs KOLKATA KNIGHT RIDERS, Qualifier 2            |League    |
|DELHI CAPITALS vs CHENNAI SUPER KINGS, Qualifier 1              |League    |
|ROYAL CHALLENGERS BENGALURU vs KOLKATA KNIGHT RIDERS, Eliminator|League    |
+----------------------------------------------------------------+----------+



In [0]:
tie_breakers = tie_breaker_matches.rdd.flatMap(lambda x: x).collect()

print("List of Tie Breakers: ")
for tie in tie_breakers:
    print(tie)

List of Tie Breakers: 
CHENNAI SUPER KINGS vs KOLKATA KNIGHT RIDERS, Final
League
DELHI CAPITALS vs KOLKATA KNIGHT RIDERS, Qualifier 2
League
DELHI CAPITALS vs CHENNAI SUPER KINGS, Qualifier 1
League
ROYAL CHALLENGERS BENGALURU vs KOLKATA KNIGHT RIDERS, Eliminator
League
